In [ ]:
%pip install matplotlib

In [ ]:
 
%pip install backtrader[plotting]


In [6]:
import backtrader as bt
import pandas as pd

class SMACrossOver(bt.Strategy):
    params = (('fast', 50), ('slow', 200),)  # Parameters for the moving averages

    def __init__(self):
        self.fast_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.fast)
        self.slow_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.slow)
        self.crossover = bt.indicators.CrossOver(self.fast_sma, self.slow_sma)

    def next(self):
        if not self.position:  # If not in the market
            if self.crossover > 0:  # If fast MA crosses above slow MA
                self.buy()  # Enter long position
        elif self.crossover < 0:  # If fast MA crosses below slow MA
            self.close()  # Close long position

if __name__ == '__main__':
    cerebro = bt.Cerebro()

    # Read data from Excel file
    df = pd.read_excel('Daily_EURUSD_Historical_Data.xlsx')

    # Convert 'Date' column to datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Convert DataFrame to Backtrader format
    data = bt.feeds.PandasData(dataname=df.set_index('Date'))

    # Add data feed
    cerebro.adddata(data)

    # Add strategy
    cerebro.addstrategy(SMACrossOver)

    # Set initial cash
    cerebro.broker.setcash(10000)

    # Set commission
    cerebro.broker.setcommission(commission=0.001)

    # Print starting cash
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run the strategy
    cerebro.run()

    # Print final portfolio value
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 10000.00
Final Portfolio Value: 10000.27


In [9]:
import backtrader as bt
import pandas as pd

class SMACrossOver(bt.Strategy):
    params = (('fast', 50), ('slow', 200),)  # Parameters for the moving averages

    def __init__(self):
        self.fast_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.fast)
        self.slow_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.slow)
        self.crossover = bt.indicators.CrossOver(self.fast_sma, self.slow_sma)

    def next(self):
        if not self.position:  # If not in the market
            if self.crossover > 0:  # If fast MA crosses above slow MA
                # Implement position sizing and risk management
                self.buy(size=self.calculate_position_size())  # Enter long position with calculated size
        elif self.crossover < 0:  # If fast MA crosses below slow MA
            self.close()  # Close long position

    def calculate_position_size(self):
        # Implement position sizing strategy (e.g., fixed fractional position sizing)
        risk_per_trade = self.broker.getvalue() * 0.01  # Risk 1% of account value per trade
        return risk_per_trade / self.data.close[0]  # Adjust position size based on current close price

if __name__ == '__main__':
    cerebro = bt.Cerebro()

    # Read data from Excel file
    df = pd.read_excel('Daily_EURUSD_Historical_Data.xlsx')

    # Convert 'Date' column to datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Convert DataFrame to Backtrader format
    data = bt.feeds.PandasData(dataname=df.set_index('Date'))

    # Add data feed
    cerebro.adddata(data)

    # Add strategy
    cerebro.addstrategy(SMACrossOver)

    # Set initial cash
    cerebro.broker.setcash(10000)

    # Set commission
    cerebro.broker.setcommission(commission=0.001)

    # Set slippage (adjust as per your broker's slippage rate)
    cerebro.broker.set_slippage_fixed(0.01)

    # Set benchmark (e.g., S&P 500 index) for benchmarking
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timereturn')
    cerebro.addanalyzer(bt.analyzers.Transactions, _name='transactions')

    # Print starting cash
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run the strategy
    strategies = cerebro.run()

    # Print final portfolio value
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Print performance metrics
    strat = strategies[0]
    print('Sharpe Ratio:', strat.analyzers.sharpe.get_analysis())
    print('Max Drawdown:', strat.analyzers.drawdown.get_analysis())
    print('Trade Analysis:', strat.analyzers.trade.get_analysis())
    print('Returns:', strat.analyzers.returns.get_analysis())
    print('Time Return:', strat.analyzers.timereturn.get_analysis())
    print('Transactions:', strat.analyzers.transactions.get_analysis())


Starting Portfolio Value: 10000.00
Final Portfolio Value: 10017.91
Sharpe Ratio: OrderedDict([('sharperatio', -13.272843197076583)])
Max Drawdown: AutoOrderedDict([('len', 3970), ('drawdown', 0.2893982707151108), ('moneydown', 29.075810016949617), ('max', AutoOrderedDict([('len', 3970), ('drawdown', 0.3980641637641281), ('moneydown', 39.993459434163015)]))])
Trade Analysis: AutoOrderedDict([('total', AutoOrderedDict([('total', 16), ('open', 1), ('closed', 15)])), ('streak', AutoOrderedDict([('won', AutoOrderedDict([('current', 0), ('longest', 2)])), ('lost', AutoOrderedDict([('current', 1), ('longest', 4)]))])), ('pnl', AutoOrderedDict([('gross', AutoOrderedDict([('total', 18.2997320553866), ('average', 1.2199821370257733)])), ('net', AutoOrderedDict([('total', 15.267986154348016), ('average', 1.017865743623201)]))])), ('won', AutoOrderedDict([('total', 5), ('pnl', AutoOrderedDict([('total', 55.76043630885492), ('average', 11.152087261770983), ('max', 31.394862939470112)]))])), ('lost'

In [16]:
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt

class SMACrossOver(bt.Strategy):
    params = (('fast', 50), ('slow', 200),)  # Parameters for the moving averages

    def __init__(self):
        self.fast_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.fast)
        self.slow_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.slow)
        self.crossover = bt.indicators.CrossOver(self.fast_sma, self.slow_sma)

    def next(self):
        if not self.position:  # If not in the market
            if self.crossover > 0:  # If fast MA crosses above slow MA
                # Implement position sizing and risk management
                self.buy(size=self.calculate_position_size())  # Enter long position with calculated size
        elif self.crossover < 0:  # If fast MA crosses below slow MA
            self.close()  # Close long position

    def calculate_position_size(self):
        # Implement position sizing strategy (e.g., fixed fractional position sizing)
        risk_per_trade = self.broker.getvalue() * 0.01  # Risk 1% of account value per trade
        return risk_per_trade / self.data.close[0]  # Adjust position size based on current close price

if __name__ == '__main__':
    cerebro = bt.Cerebro()

    # Read data from Excel file
    df = pd.read_excel('Daily_EURUSD_Historical_Data.xlsx')

    # Convert 'Date' column to datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Convert DataFrame to Backtrader format
    data = bt.feeds.PandasData(dataname=df.set_index('Date'))

    # Add data feed
    cerebro.adddata(data)

    # Add strategy
    cerebro.addstrategy(SMACrossOver)

    # Set initial cash
    cerebro.broker.setcash(10000)

    # Set commission
    cerebro.broker.setcommission(commission=0.001)

    # Print starting cash
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run the strategy
    cerebro.run()

    # Print final portfolio value
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Plot the strategy's results (candlestick chart)
    cerebro.plot(style='candlestick', volume=False)
    plt.show()


Starting Portfolio Value: 10000.00
Final Portfolio Value: 10028.73


<IPython.core.display.Javascript object>